In [1]:
import numpy as np
import glob
import random
from segment2d import *
import nibabel as nib
from ipywidgets import interact
import matplotlib.pyplot as plt
import albumentations as A
from torchvision import transforms
import os

import csv
import kornia as K
from tqdm import tqdm
from torch.utils.data import DataLoader
import yaml
import SimpleITK as sitk


In [ ]:
# flair_full = np.zeros((len(list_file_flair), 240, 240, 155))
# t1_full = np.zeros((len(list_file_flair), 240, 240, 155))
# t2_full = np.zeros((len(list_file_flair), 240, 240, 155))
# for count_subject, path_flair in tqdm(enumerate(list_file_flair)):

#     flair = nib.load(path_flair).get_fdata()
#     t1 = nib.load(path_flair.replace("flair", "t1")).get_fdata()
#     t2 = nib.load(path_flair.replace("flair", "t2")).get_fdata()
#     # caculate mean std for all data
    
#     flair = min_max_normalize(flair)
#     t1 = min_max_normalize(t1)
#     t2 = min_max_normalize(t2)
    
#     flair_full[count_subject] = flair
#     t1_full[count_subject] = t1
#     t2_full[count_subject] = t2

In [ ]:
# mean_flair, std_flair = np.mean(flair_full[flair_full != 0]), np.std(flair_full[flair_full != 0])
# mean_t1, std_t1 = np.mean(t1_full[t1_full != 0]), np.std(t1_full[t1_full != 0])
# mean_t2, std_t2 = np.mean(t2_full[t2_full != 0]), np.std(t2_full[t2_full != 0])
# dict_mean_std = {}
# dict_mean_std["mean_flair"] = mean_flair.tolist()
# dict_mean_std["mean_t2"] = mean_t2.tolist()
# dict_mean_std["mean_t1"] = mean_t1.tolist()
# dict_mean_std["std_flair"] = mean_t2.tolist()
# dict_mean_std["std_t2"] = std_t2.tolist()
# dict_mean_std["std_t1"] = std_t1.tolist()
# with open('mean_std_brats_norm.yaml', 'w') as outfile:
#     # Write the dictionary to the file
#     yaml.dump(dict_mean_std, outfile)

In [5]:
list_of_flair = glob.glob("data/msseg-2008-training-nii/*/*FLAIR_brain.nii.gz")
for flair_dir in list_of_flair:
    flair = nib.load(flair_dir).get_fdata()
    mask = nib.load(flair_dir.replace("FLAIR_brain", "lesion")).get_fdata()
    # padded_flair, crop_index, padded_index = pad_background(flair, dim2pad=(320,320,320))

    # padded_mask = pad_background_with_index(mask, crop_index, padded_index, dim2pad=(320,320,320))
    padded_flair = flair[96:416,96:416,96:416]
    padded_mask = mask[96:416,96:416,96:416]
    if np.sum(padded_mask)-np.sum(mask) != 0:
        print(np.sum(padded_mask)-np.sum(mask))
        print(flair_dir)

In [2]:
flair_dir = glob.glob("data/msseg-2008-training-nii/*/*FLAIR_brain.nii.gz")[-1]

flair = nib.load(flair_dir).get_fdata()
mask = nib.load(flair_dir.replace("FLAIR_brain", "lesion")).get_fdata()

In [3]:
#get the center indice with shape 320,320,320 from shape 512,512,512
padded_flair = flair[96:416,96:416,96:416]
padded_mask = mask[96:416,96:416,96:416]

In [4]:
# flair = nib.load("data/data_isbi_2015/training/training01/preprocessed/training01_01_flair_pp.nii").get_fdata()
# aug_flair = augmentation(image=image_flair)["image"]
def show_segmentation(index):
    try:
        plt.figure(figsize=(20, 20))
        plt.subplot(1, 5, 1),
        plt.imshow(padded_flair[index]),
        plt.subplot(1, 5, 2),
        plt.imshow(flair[index]),
        plt.subplot(1, 5, 3),
        plt.imshow(mask[index]),
        plt.subplot(1, 5, 4),
        plt.imshow(padded_mask[index]),
        plt.show(),
    except:
        pass
    # plt.subplot(1, 5, 5),
    # plt.imshow(augmented_mask),
interact(show_segmentation, index=(0,500))

interactive(children=(IntSlider(value=250, description='index', max=500), Output()), _dom_classes=('widget-int…

<function __main__.show_segmentation(index)>

In [ ]:
import os
import subprocess
import nibabel as nib
import SimpleITK as sitk
import ants
import ants.utils.bias_correction as ants_bias
from tqdm import tqdm
import numpy as np
# this file is used to convert the msseg2008 dataset to nifti format
# then remove brain skull based on t1 weighted then filter to T2 and FLAIR
# correct bias field
# sub_data = "training" or "testing"
# the final dataset is stored in data/msseg-2008-{sub_data}-nii
# the original dataset is stored in data/msseg-2008-{sub_data}
# it requires fsl and antspyx to be installed

FSLDIR = os.environ.get('FSLDIR')

datasets = ["training", "testing"]

for sub_data in datasets:
    PATH = f'data/msseg-2008-{sub_data}/'
    dir_list = []
    file_list = []
    for folder in os.listdir(PATH):
        dir_list.append(folder)
    for folder in dir_list:
        folder_path = os.path.join(PATH, folder)
        for image in os.listdir(folder_path):
            if image.endswith(".nhdr"):
                file_list.append((folder, image[:-5]))
    
    print(f"remove brain skull and save to folder {sub_data} with _brain.nii.gz ...")

    for image in tqdm(file_list):
        (folder, file) = image
        img = sitk.ReadImage(f'{PATH}/{folder}/{file}.nhdr')
        newfile = file + ".nii.gz"
        NEW_PATH = f'data/msseg-2008-{sub_data}-nii/{folder}'
        if not os.path.exists(NEW_PATH):
            os.makedirs(NEW_PATH) 
        sitk.WriteImage(img, f'data/msseg-2008-{sub_data}-nii/{folder}/{newfile}')
        if "T1" in file:
            command = [f"{FSLDIR}/bin/bet", f'data/msseg-2008-{sub_data}-nii/{folder}/{file}', f'data/msseg-2008-{sub_data}-nii/{folder}/{file}_brain', "-f","0.5","-g","0"]
            subprocess.run(command, check = True)
            

In [ ]:
cfg.TRAIN.FOLD =5
list_file_flair = sorted(glob.glob(f"data/MSSEG-Training_2016/Training/Center*/*/Preprocessed_Data/*FLAIR*"))
list_file_flair

In [ ]:
flair = nib.load(list_file_flair[0]).get_fdata()
mask = nib.load(list_file_flair[0].replace("Preprocessed_Data", "Masks").replace("FLAIR_preprocessed","Consensus")).get_fdata()

In [ ]:
list_file_flair = sorted(glob.glob(f"data/msseg-2008-training/*"))
len(list_file_flair)

In [8]:
a

In [7]:
import nibabel as nib
a = nib.load("UNC_train_Case01_T1.nii.gz")
ants.n4_bias_field_correction(a)

AttributeError: 'Nifti1Image' object has no attribute 'pixeltype'

In [9]:
img

ANTsImage (RAI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (512, 512, 512)
	 Spacing    : (0.5, 0.5, 0.5)
	 Origin     : (0.0, 0.0, 0.0)
	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.]

In [ ]:
path_flair = list_file_flair[10]
flair = nib.load(path_flair).get_fdata()
t1 = nib.load(path_flair.replace("FLAIR", "T1")).get_fdata()
t2 = nib.load(path_flair.replace("FLAIR", "T2")).get_fdata()

consensus = nib.load(path_flair.replace("Preprocessed_Data", "Masks").replace("FLAIR_preprocessed", "Consensus"))
consensus = consensus.get_fdata().astype(np.uint8)

padded_flair, crop_index, padded_index = pad_background(flair, dim2pad=cfg.DATA.DIM2PAD_MICCAI)
padded_t1 = pad_background_with_index(t1, crop_index, padded_index)
padded_t2 = pad_background_with_index(t2, crop_index, padded_index)
padded_mask = pad_background_with_index(consensus, crop_index, padded_index)

In [ ]:
padding_t1.shape

In [ ]:
padded_t2.shape

In [ ]:
len(list_file_flair)

In [ ]:
list_file_flair = sorted(glob.glob(f"data/MSSEG-Training_2016/Training/Center*/*/Preprocessed_Data/*FLAIR*"))
list_file_flair = sorted(glob.glob(f"data/MSSEG-Testing/Center*/*/Preprocessed_Data/*FLAIR*"))

In [ ]:
# list_file_flair


In [ ]:
for i in range(len(list_file_flair)):
    flair = nib.load(list_file_flair[i]).get_fdata()
    mask = nib.load(list_file_flair[i].replace("Preprocessed_Data", "Masks").replace("FLAIR_preprocessed","Consensus")).get_fdata()
    # padded_image, [padded_mask], crop_index = pad_background(flair, [mask])
    if np.sum(mask) == 0:
        print(i)

In [ ]:
image = flair
padded_image, crop_index, padded_index = pad_background(image)
padded_image, [padded_mask], crop_index = pad_background(image, [mask])

# perform some operations on the padded image
orig_image = invert_padding(image, padded_image, crop_index, padded_index)

In [ ]:
np.sum(padded_mask)-np.sum(mask)

In [ ]:
np.linalg.norm(mask[crop_index]-padded_mask[crop_index])


In [ ]:
np.linalg.norm(flair[crop_index]-orig_image[crop_index])

In [ ]:
# invert = np.zeros_like(original_flair)
# invert[crop_index] = padded_image[padded_index]
np.linalg.norm(orig_image-original_flair)

In [6]:
# flair = nib.load("data/data_isbi_2015/training/training01/preprocessed/training01_01_flair_pp.nii").get_fdata()
# aug_flair = augmentation(image=image_flair)["image"]
def show_segmentation(index):
    try:
        plt.figure(figsize=(20, 20))
        plt.subplot(1, 5, 1),
        plt.imshow(padded_flair[...,index]),
        plt.subplot(1, 5, 2),
        plt.imshow(flair[...,index]),
        # plt.subplot(1, 5, 3),
        # plt.imshow(padded_t2[...,index]),
        # plt.subplot(1, 5, 4),
        # plt.imshow(padded_mask[...,index]),
        plt.show(),
    except:
        pass
    # plt.subplot(1, 5, 5),
    # plt.imshow(augmented_mask),
interact(show_segmentation, index=(0,500))

interactive(children=(IntSlider(value=250, description='index', max=500), Output()), _dom_classes=('widget-int…

<function __main__.show_segmentation(index)>

In [ ]:
flair = np.load("data/msseg2016npz/mri1_axial_50_mask1.npz")["flair"]
mask = np.load("data/msseg2016npz/mri1_axial_50_mask1.npz")["mask"]


In [ ]:
flair.shape

In [ ]:
image = torch.from_numpy(flair.transpose(-1, 0, 1))[None]
mask = torch.from_numpy(mask[None].astype(np.float32))[None]

In [ ]:
np.linalg.norm(imgs_aug_image-flair)

In [ ]:

# Perform basic data augmentation using Kornia.
from kornia.filters import GaussianBlur2d
aug = K.augmentation.AugmentationSequential(
    GaussianBlur2d((3, 3), (1.5, 1.5)),
    # K.augmentation.ColorJitter((0.9, 1.1), (0.9, 1.1) , p=1),
    # K.augmentation.RandomVerticalFlip(p=1),
    # K.augmentation.RandomAffine(degrees=10, translate=0.0625, scale=(0.95, 1.05), p=1.0,),
    # K.augmentation.RandomBrightness((1, 1.), p=1),
    # data_keys=["input", "mask"]
)
imgs_aug, mask_aug = aug(image), mask
imgs_aug_image = K.tensor_to_image(imgs_aug)
mask_aug_image = K.tensor_to_image(mask_aug)

def show_segmentation(index):
    plt.figure(figsize=(12, 12))
    plt.subplot(1, 5, 1),
    plt.imshow(flair[..., index]),
    plt.subplot(1, 5, 2),
    plt.imshow(mask[0,0]),
    plt.subplot(1, 5, 3),
    plt.imshow(imgs_aug_image[..., index]),
    plt.subplot(1, 5, 4),
    plt.imshow(mask_aug_image),
    # plt.subplot(1, 5, 5),
    # plt.imshow(augmented_mask),
    plt.show(),
interact(show_segmentation, index=(0, flair.shape[-1]-1))

In [ ]:
import SimpleITK as sitk
img = sitk.ReadImage('data/msseg-2008-training/UNC_train_Case02/UNC_train_Case02_FLAIR.nhdr')
sitk.WriteImage(img, 'UNC_train_Case01_FLAIR.nii.gz')

img = sitk.ReadImage('data/msseg-2008-training/UNC_train_Case02/UNC_train_Case02_T1.nhdr')
sitk.WriteImage(img, 'UNC_train_Case01_T1.nii.gz')

img = sitk.ReadImage('data/msseg-2008-training/UNC_train_Case02/UNC_train_Case02_T2.nhdr')
sitk.WriteImage(img, 'UNC_train_Case01_T2.nii.gz')

img = sitk.ReadImage('data/msseg-2008-training/UNC_train_Case02/UNC_train_Case02_lesion_byCHB.nhdr')
sitk.WriteImage(img, 'UNC_train_Case01_lesion_byCHB.nii.gz')

In [ ]:
list_of_train = sorted(glob.glob("data/msseg-2008-training/*/*lesion*nhdr"))
len(list_of_train)

In [ ]:
list_of_train

In [ ]:
import nibabel.processing as nib_pro
t1 = nib.load("UNC_train_Case01_T1.nii.gz")
t1_2 = nib_pro.conform(t1, out_shape=(256, 256, 256), voxel_size=(1.0, 1.0, 1.0), orientation="LPS")

In [ ]:
# save t1_2 to nii.gz
nib.save(t1_2, "UNC_train_Case01_T1_2.nii.gz")

In [ ]:
# read numpy data from SimpleITK.SimpleITK.Image


mask = sitk.GetArrayFromImage(sitk.ReadImage('data/msseg-2008-training/UNC_train_Case02/UNC_train_Case02_lesion_byCHB.nhdr'))
t1 = sitk.GetArrayFromImage(sitk.ReadImage('data/msseg-2008-training/UNC_train_Case02/UNC_train_Case02_T1.nhdr'))
# crop the center of the image t1 with size of 256 256 256
for x in range(512):
    if np.sum(mask) - np.sum(mask[x:x+256, x:x+256, x:x+256]) == 0:
        print(x)
        break



In [ ]:
np.sum(mask) - np.sum(mask[x:x+256, x:x+256, x:x+256])

In [ ]:
def show_segmentation(index):
    plt.figure(figsize=(12, 12))
    plt.subplot(1, 5, 1),
    plt.imshow(padded_t1[:,:,index]),
    plt.subplot(1, 5, 2),
    plt.imshow(t1[:,:,index]),
    # plt.subplot(1, 5, 3),
    # plt.imshow(original_flair[:,index,:]),
    # plt.subplot(1, 5, 4),
    # plt.imshow(original_mask1[:,index,:]),
    # plt.subplot(1, 5, 5),
    # plt.imshow(augmented_mask),
    plt.show(),
interact(show_segmentation, index=(0, t1.shape[0]-1))

In [ ]:
a = nib.load("predict_testset/test01_01_nhatvin.nii").get_fdata()
np.unique(a)

In [ ]:
original_flair = nib.load("data/data_isbi_2015/training/training01/orig/training01_01_flair.nii.gz").get_fdata()
original_mask1 = nib.load("data/data_isbi_2015/training/training01/masks/training01_01_mask2.nii").get_fdata()
# original_flair, original_mask1 = pad_remove_background(original_flair, [original_mask1], dim2pad=cfg.DATA.DIM2PAD_ISBI)
# original_mask1 = original_mask1[0]

In [ ]:
a = np.load("data/data_isbi_2015/isbi2npz2D/mri1_axial_80_mask2.npz")
a["flair"].min()

In [ ]:
a["flair"].shape

In [ ]:

subject = np.load("data/data_isbi_2015/isbi2npz2D/mri1_axial_74_mask1.npz")
flair, mask = subject['flair'], subject['mask']

In [ ]:
torch.from_numpy(flair).shape

In [ ]:
    nz = np.nonzero(b)

    # get the minimum and maximum indices along each axis
    min_indices = np.min(nz, axis=1)
    max_indices = np.max(nz, axis=1)

    # crop the image to only include non-zero values
    crop_index = tuple(slice(imin, imax+1) for imin, imax in zip(min_indices, max_indices))

In [ ]:
b[crop_index].shape

In [ ]:
original_flair = nib.load("data/data_isbi_2015/training/training01/preprocessed/training01_01_flair_pp.nii").get_fdata()
original_mask1 = nib.load("data/data_isbi_2015/training/training01/masks/training01_01_mask2.nii").get_fdata()
def show_segmentation(index):
    plt.figure(figsize=(12, 12))
    plt.subplot(1, 5, 1),
    plt.imshow(original_flair[:,index,:]),
    plt.subplot(1, 5, 2),
    plt.imshow(original_mask1[:,index,:]),
    # plt.subplot(1, 5, 3),
    # plt.imshow(original_flair[:,index,:]),
    # plt.subplot(1, 5, 4),
    # plt.imshow(original_mask1[:,index,:]),
    # plt.subplot(1, 5, 5),
    # plt.imshow(augmented_mask),
    plt.show(),
interact(show_segmentation, index=(0, original_flair.shape[0]-1))

In [ ]:
x = np.random.randint(0, 100, size=(224, 224, 224))

In [ ]:
x1 = x.transpose((1, 2, 0))
x2 = np.expand_dims(x1, axis=-1)
x2 = x2.transpose((2, 0, 1, -1))

np.linalg.norm(x - x2[...,0])

In [ ]:
a = np.load("predict_testset/training05_01_view1.npy")
b = np.load("predict_testset/training05_01_view2.npy")
c = np.load("predict_testset/training05_01_view3.npy")
seg1 = a+b+c
seg1 = np.argmax(seg1, axis=-1)
a =  np.argmax(a, axis=-1)
b =  np.argmax(b, axis=-1)
c =  np.argmax(c, axis=-1)
seg2 = a+b+c
seg2 = (seg2>=1).astype(int)

In [ ]:
mask2 = nib.load("data/data_isbi_2015/training/training05/masks/training05_01_mask1.nii").get_fdata()
print(dice_MS_volume(a, mask2))
print(dice_MS_volume(b, mask2))
print(dice_MS_volume(c, mask2))
print(dice_MS_volume(seg1, mask2))
print(dice_MS_volume(seg2, mask2))

In [ ]:
mask2 = nib.load("data/data_isbi_2015/training/training05/masks/training05_03_mask1.nii").get_fdata()
print(dice_MS_volume(a, mask2))
print(dice_MS_volume(b, mask2))
print(dice_MS_volume(c, mask2))
print(dice_MS_volume(seg1, mask2))
print(dice_MS_volume(seg2, mask2))


In [ ]:
flair = nib.load(list_file_flair[0]).get_fdata()
flair.shape

In [ ]:
# train on axial a

In [ ]:
a, valid_index = pad_remove_background(flair)
a.shape

In [ ]:
print(list_file_flair[0:1])
print(checkpoint)

In [ ]:
cfg.DATA.ORIGIN2CUT = [(1, 2, 0), (0, 2, 1), (0, 1, 2)]
cfg.DATA.CUT2ORIGIN = [(2, 0, 1), (0, 2, 1), (0, 1, 2)]

In [ ]:

imagex = flair.transpose(cfg.DATA.ORIGIN2CUT[0])


In [ ]:
image_x = imagex[..., 50:51]

In [ ]:
np.count_nonzero(mask1[..., 42])

In [ ]:
flair = np.load("data/data_isbi_2015/isbi2npz2D/mri1_coronal_100_mask2.npz")["flair"]
mask = np.load("data/data_isbi_2015/isbi2npz2D/mri1_coronal_100_mask2.npz")["mask"]
def show_segmentation(index):
    plt.figure(figsize=(12, 12))
    plt.subplot(1, 5, 1),
    plt.imshow(flair[...,index]),
    plt.subplot(1, 5, 2),
    plt.imshow(mask),
    # plt.subplot(1, 5, 3),
    # plt.imshow(c[index]),
    # plt.subplot(1, 5, 4),
    # plt.imshow(mask2[index,:]),
    # plt.subplot(1, 5, 5),
    # plt.imshow(augmented_mask),
    plt.show(),
interact(show_segmentation, index=(0, 3))

In [ ]:
seg.shape

In [ ]:
def getValidIndex(volume, size=(160, 192, 155)):
    """ this function remove unnescessary background and center crop remaining part """
    x_indexes, y_indexes, z_indexes = np.nonzero(np.sum(volume, axis=0) != 0)
    dims_min = np.min([x_indexes, y_indexes, z_indexes], axis=1)
    dims_max = np.max([x_indexes, y_indexes, z_indexes], axis=1)

    dims_min = dims_min + (dims_max - dims_min - size) // 2
    dims_min[dims_min < 0] = 0
    dims_min[-1] = 0 # because size[-1] always is larger than true size, remove this line for another task.
    dims_max = dims_min + size
    return tuple(slice(imin, imax) for imin, imax in zip(dims_min, dims_max))

In [ ]:
e = []
a = np.ones((4, 2, 2, 2))
b = np.ones((4, 2, 2, 2))
c= np.ones((4, 2, 2, 2))
d = np.ones((4, 2, 2, 2))

In [ ]:
e.append(a)
e.append(b)
e.append(c)
e.append(d)
e = np.mean(np.stack(e, axis=0), axis=0)

In [ ]:
np.linalg.norm(e-a)

In [ ]:
mask1 = nib.load(path_flair.replace("flair_pp", "mask1").replace("preprocessed", "masks")).get_fdata()
mask2 = nib.load(path_flair.replace("flair_pp", "mask2").replace("preprocessed", "masks")).get_fdata()

mask = np.logical_or(mask1, mask2).astype(np.int64)

In [ ]:
output = np.argmax(y_pred, axis=1).transpose(1, 2, 0)
output.shape

In [ ]:
output[valid_index].shape

In [ ]:
seg.shape

In [ ]:
seg = np.zeros_like(flair)
seg[..., 1:-1] = output[valid_index]